In [ ]:
from datetime import datetime, timedelta

def get_minMax_dates(date_string):
    """
    Dado el titulo de un archivo de csv de bicing, extrae las fechas límite del mes en datetime
    """
    # String de ejemplo
    # date_string = "2020_12_Abril_BicingNou_ESTACIONS"

    # Separar el año y el mes del string
    year, month_str, *_ = date_string.split('_')
    month = datetime.strptime(month_str, "%m").month

    # Crear objeto datetime para el primer día del mes de ese año
    start_date = datetime(int(year), month, 1)

    # Calcular el primer día del mes siguiente
    next_month_start_date = start_date.replace(day=1, month=start_date.month % 12 + 1, year=start_date.year + start_date.month // 12)

    # Restar una hora para obtener el último día del mes de ese año
    end_date = next_month_start_date - timedelta(hours=1)

    return(start_date, end_date)


In [ ]:
import pandas as pd
import os

# Definir la subcarpeta
#subfolder = '../../data_csv_bicing/'
subfolder = 'data/'

# Obtener la lista de archivos CSV en la subcarpeta
files = os.listdir(subfolder)
files = [f for f in files if f.endswith('ESTACIONS.csv')]


# Lista para almacenar los DataFrames de cada archivo CSV
df_list = []

# Leer cada archivo CSV y añadirlo a la lista
for file in files:

    # Obtener fechas limite del fichero
    start_date, end_date = get_minMax_dates(file)

    # Leemos el csv y creamos un DataFrame
    file_path = os.path.join(subfolder, file)
    df_temp = pd.read_csv(file_path)

    # Convertir el campo 'last_reported' del DataFrame a objetos datetime
    df_temp['last_reported'] = pd.to_datetime(df_temp['last_reported'], unit='s')

    # Filtrar el DataFrame para mantener solo las filas dentro del rango de fechas
    df_filtered = df_temp[(df_temp['last_reported'] >= start_date) & (df_temp['last_reported'] <= end_date)].copy()

    # Ahora df_filtered contiene solo las filas con 'last_reported' dentro del rango de fechas especificado
    df_list.append(df_filtered)

# Concatenar todos los DataFrames en uno solo
big_df = pd.concat(df_list, ignore_index=True)

# Guardar en formato parquet
big_df.to_parquet('data/1_all_data_raw.heavy.parquet', index=False)


In [1]:
import pandas as pd
import numpy as np

# Leer el archivo .parquet 
df_raw = pd.read_parquet('data/1_all_data_raw.heavy.parquet')

# Cargar datos de las estaciones
df_station_info = pd.read_csv('data/Informacio_Estacions_Bicing.csv')

In [2]:
# limpiar los datos
 
# Eliminar filas donde 'station_id' es nulo y pasarlos a int
df_raw = df_raw.dropna(subset=['station_id'])
df_raw['station_id'] = df_raw['station_id'].astype(int)

# seleccionar columnas relevantes
cols_to_keep = ['station_id', 'num_bikes_available', 'num_bikes_available_types.mechanical', 'num_bikes_available_types.ebike','num_docks_available', 'last_reported']
df_raw = df_raw[cols_to_keep]

# se tienen que eliminar filas con numeros de station_id inexistentes en el registro de estaciones.
df_merge = df_raw.merge(df_station_info[["station_id", "capacity"]],
                             on="station_id",
                             how="inner")

# Timestamp to hour, day, month, year
df_merge['hour'] = df_merge['last_reported'].dt.hour
df_merge['day'] = df_merge['last_reported'].dt.day
df_merge['month'] = df_merge['last_reported'].dt.month
df_merge['year'] = df_merge['last_reported'].dt.year


# Guardar en formato parquet
df_merge.to_parquet("data/2_all_data_mean_hour.heavy.parquet", index=False)

In [20]:
import pandas as pd
import numpy as np

# Leer el archivo .parquet 
df_merge = pd.read_parquet('data/2_all_data_mean_hour.heavy.parquet')

In [21]:
df_merge

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,capacity,hour,day,month,year
0,19,25.0,23.0,2.0,2.0,2020-06-01 00:00:36,30,0,1,6,2020
1,22,10.0,10.0,0.0,9.0,2020-06-01 00:00:01,19,0,1,6,2020
2,28,7.0,5.0,2.0,16.0,2020-06-01 00:00:01,23,0,1,6,2020
3,54,16.0,15.0,1.0,0.0,2020-06-01 00:00:11,19,0,1,6,2020
4,66,0.0,0.0,0.0,12.0,2020-06-01 00:00:29,16,0,1,6,2020
...,...,...,...,...,...,...,...,...,...,...,...
196088056,515,5.0,5.0,0.0,19.0,2020-09-30 21:52:58,24,21,30,9,2020
196088057,516,18.0,16.0,2.0,3.0,2020-09-30 21:50:54,21,21,30,9,2020
196088058,517,3.0,3.0,0.0,20.0,2020-09-30 21:54:16,20,21,30,9,2020
196088059,518,2.0,0.0,2.0,24.0,2020-09-30 21:52:40,27,21,30,9,2020


In [22]:
# Verificar si hay alguna columna relevaante con valores nulos
df_merge.isnull().any()



station_id                              False
num_bikes_available                     False
num_bikes_available_types.mechanical    False
num_bikes_available_types.ebike         False
num_docks_available                     False
last_reported                           False
capacity                                False
hour                                    False
day                                     False
month                                   False
year                                    False
dtype: bool

In [23]:
# Merge by taking the mean of the values
df_merge = df_merge.groupby(['station_id', 'hour', 'day', 'month', 'year']).mean().reset_index()

# volvemos a poner la fecha correcta
df_merge['last_reported'] = pd.to_datetime(df_merge[['year', 'month', 'day', 'hour']])

# Creamos el porcentaje de slots disponibles en cada estacion en cada momento
df_merge["percentage_docks_available"] = df_merge["num_docks_available"] / df_merge["capacity"]
df_merge.to_parquet("data/3_all_data_ctx.heavy.parquet", index=False)


In [10]:
import pandas as pd

# Leer el archivo .parquet 
df_ctx = pd.read_parquet('data/3_all_data_ctx.heavy.parquet')

In [11]:
df_ctx

,station_id,hour,day,month,year,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,capacity,percentage_docks_available
0,1,0,1,1,2020,23.333333,23.166667,0.166667,20.666667,2020-01-01 00:00:00,45.0,0.459259
1,1,0,1,1,2021,40.500000,40.500000,0.000000,5.500000,2021-01-01 00:00:00,45.0,0.122222
2,1,0,1,1,2022,14.583333,12.583333,2.000000,31.416667,2022-01-01 00:00:00,45.0,0.698148
3,1,0,1,1,2023,39.333333,36.666667,2.666667,6.666667,2023-01-01 00:00:00,45.0,0.148148
4,1,0,1,2,2020,31.500000,30.083333,1.416667,12.500000,2020-02-01 00:00:00,45.0,0.277778
...,...,...,...,...,...,...,...,...,...,...,...,...
16349587,519,23,30,10,2023,16.166667,6.000000,10.166667,6.833333,2023-10-30 23:00:00,24.0,0.284722
16349588,519,23,30,12,2020,9.000000,0.000000,9.000000,15.000000,2020-12-30 23:00:00,24.0,0.625000
16349589,519,23,30,12,2021,5.416667,0.000000,5.416667,18.583333,2021-12-30 23:00:00,24.0,0.774306
16349590,519,23,30,12,2022,2.833333,0.000000,2.833333,21.166667,2022-12-30 23:00:00,24.0,0.881944


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta

## Este codigo garantiza que estan todas las fechas.

# seleccionar columnas relevantes
cols_to_keep = ['station_id', 'year', 'month', 'day', 'hour', 'percentage_docks_available']
df_ctx = df_ctx[cols_to_keep]

# Lista de rows finales
rows = []

# para cada station_id
for st_id in tqdm(df_ctx.station_id.unique()):
    # creamos df ordenado por fecha.        
    df_station = df_ctx[df_ctx['station_id'] == st_id].sort_values(by=["year", "month", "day", "hour"])
    
    # Obtenemos la fecha mayor y menor
    min_row = df_station.iloc[0]
    min_data = datetime(int(min_row['year']), int(min_row['month']), int(min_row['day']), int(min_row['hour']))
    max_row = df_station.iloc[-1]
    max_data = datetime(int(max_row['year']), int(max_row['month']), int(max_row['day']), int(max_row['hour']))

    # empezamos con la fecha mas antigua
    current_date = min_data
    
    # Para cada registro
    for current_row in df_station.itertuples(index=False):
        current_row_date = datetime(current_row.year, current_row.month, current_row.day, current_row.hour)
        
        # mientras no concida la fecha, añadimos registro faltante
        while current_date < current_row_date:

            new_row = {
                'station_id': st_id,
                'year': current_date.year,
                'month': current_date.month,
                'day': current_date.day,
                'hour': current_date.hour,
                'percentage_docks_available': np.nan
            }
            rows.append(new_row)
            # Sumamos una hora 
            current_date += timedelta(hours=1)
        
        # Llegar aqui implica que la fecha coincide
        rows.append(current_row._asdict())

        # Sumamos una hora 
        current_date += timedelta(hours=1)

# Convertimos la lista de diccionarios en un DataFrame
df_complet = pd.DataFrame(rows)

df_complet.to_parquet("data/4_all_dates_null_ctx.parquet", index=False)


In [17]:
import pandas as pd

# Leer el archivo .parquet 
df_dates = pd.read_parquet('data/4_all_dates_null_ctx.parquet')

In [18]:
df_dates

,station_id,year,month,day,hour,percentage_docks_available
0,1,2020,1,1,0,0.459259
1,1,2020,1,1,1,0.394444
2,1,2020,1,1,2,0.346296
3,1,2020,1,1,3,0.283333
4,1,2020,1,1,4,0.364815
...,...,...,...,...,...,...
17371817,519,2023,12,31,18,0.868056
17371818,519,2023,12,31,19,0.767361
17371819,519,2023,12,31,20,0.715278
17371820,519,2023,12,31,21,0.684028


In [23]:
from tqdm import tqdm

rows = []
# para cada station_id
for st_id in tqdm(df_dates.station_id.unique()):
    # creamos df ordenado por fecha.
    df_station = df_dates[df_dates['station_id'] == st_id].sort_values(by=["year", "month", "day", "hour"])

    # Para cada registro
    # Iterar sobre las filas a partir de la quinta fila y saltando de 5 en 5
    for i in range(4, len(df_station), 5):

        tmp_row = df_station.iloc[i].copy()
        # retrocedemos las 4 horas anteriores para guardar sus valores si los tiene
        for j in range(1,5):
                # Buscamos la hora y su registro
                tmp_row[f'ctx-{j}'] = df_station.iloc[i-j].percentage_docks_available
        # añadimos el registro con los 4 ctx anteriores.
        rows.append(tmp_row.to_dict())

# Convertimos la lista de diccionarios en un DataFrame
prediction_data = pd.DataFrame(rows)

prediction_data.to_parquet("data/5_compress_dates_null_ctx_4h.parquet", index=False)

100%|██████████| 506/506 [31:21<00:00,  3.72s/it]


In [24]:
import pandas as pd

# Leer el archivo .parquet 
df_ctx4 = pd.read_parquet('data/5_compress_dates_null_ctx_4h.parquet')

In [29]:
# Verificar % de nulos por columna
df_ctx4.isnull().sum()/len(df_ctx4)*100

station_id                    0.000000
year                          0.000000
month                         0.000000
day                           0.000000
hour                          0.000000
percentage_docks_available    5.971015
ctx-1                         5.897614
ctx-2                         5.900579
ctx-3                         5.798163
ctx-4                         5.857085
dtype: float64